In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from config import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.32,74,89,12.38,PN,1713771336
1,1,puerto ayora,-0.7393,-90.3518,24.19,99,80,1.34,EC,1713771347
2,2,margaret river,-33.9500,115.0667,18.03,42,10,7.88,AU,1713770933
3,3,segou,13.4317,-6.2157,30.77,46,37,7.43,ML,1713771367
4,4,remire-montjoly,4.9167,-52.2667,25.02,94,75,1.03,GF,1713771377


In [20]:
#Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [22]:
#Narrow down the city_data_df DataFrame to find your ideal weather condition

In [23]:

# Narrow down cities that fit criteria and drop any results with null values

# My Criteria:
# A max temperature lower than 25 degrees but higher than 20
# Cloudiness less than 2
# Wind speed less than 3 m/s


weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 25)
                                       & (city_data_df["Max Temp"] > 20) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] < 3)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
236,236,omerli,37.4022,40.9541,20.24,31,0,1.61,TR,1713773928
273,273,melchor de mencos,17.0686,-89.1522,24.00,88,0,0.00,GT,1713774335
362,362,coahuayana de hidalgo,18.7000,-103.6583,21.15,82,1,1.19,MX,1713775312
377,377,mchinji,-13.7984,32.8802,24.96,50,0,1.93,MW,1713775466
435,435,cabo san lucas,22.8909,-109.9124,24.73,49,0,0.89,MX,1713776094
474,474,les anglais,18.3054,-74.2197,24.39,83,0,1.92,HT,1713776519
512,512,ilhabela,-23.7781,-45.3581,20.36,81,0,1.20,BR,1713776945
536,536,san luis de la loma,17.2706,-100.8939,23.93,61,0,0.57,MX,1713777214


In [24]:
#Create a new DataFrame called hotel_df.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
236,omerli,TR,37.4022,40.9541,31,
273,melchor de mencos,GT,17.0686,-89.1522,88,
362,coahuayana de hidalgo,MX,18.7000,-103.6583,82,
377,mchinji,MW,-13.7984,32.8802,50,
435,cabo san lucas,MX,22.8909,-109.9124,49,
474,les anglais,HT,18.3054,-74.2197,83,
512,ilhabela,BR,-23.7781,-45.3581,81,
536,san luis de la loma,MX,17.2706,-100.8939,61,


In [26]:
#For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
omerli - nearest hotel: No hotel found
melchor de mencos - nearest hotel: Hotel Central
coahuayana de hidalgo - nearest hotel: No hotel found
mchinji - nearest hotel: Greenhill Lodge
cabo san lucas - nearest hotel: Comfort Rooms
les anglais - nearest hotel: No hotel found
ilhabela - nearest hotel: Vila Kebaya
san luis de la loma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
236,omerli,TR,37.4022,40.9541,31,No hotel found
273,melchor de mencos,GT,17.0686,-89.1522,88,Hotel Central
362,coahuayana de hidalgo,MX,18.7000,-103.6583,82,No hotel found
377,mchinji,MW,-13.7984,32.8802,50,Greenhill Lodge
435,cabo san lucas,MX,22.8909,-109.9124,49,Comfort Rooms
474,les anglais,HT,18.3054,-74.2197,83,No hotel found
512,ilhabela,BR,-23.7781,-45.3581,81,Vila Kebaya
536,san luis de la loma,MX,17.2706,-100.8939,61,No hotel found


In [28]:
#Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)